# Húsnæðislánareiknivél

Þetta er lauflétt húsnæðislánareiknivél sem ég bjó til til þess að skilja betur hvernig húsnæðislán virka.

Áður en þú byrjar að nota reiknivélina þarftu að setja upp þau forritasöfn sem eru nauðsynleg. Það er gert með því að keyra `python -m pip install -r requirements.txt`. Þessi skrifta hefur verið prófuð á Python útgáfu 3.11.2.

Til að nota reiknivélina er nóg breyta breytunum í næsta hólfi (Stillingar) og keyra reiknivélina (Run all).

In [1]:
# Stillingar - Hægt að breyta þessum gildum

# Lán
lansupphaed: int = 20000000
lanstimi: int = 40 # í árum
vextir: float = 9.25 # Ársvextir í prósentum

verdtrygging: bool = False
verdbolga: float = 8.00 # Áætluð árleg verðbólga í prósentum

jafnar_afborganir: bool = True # Ef False þá er getrt ráð fyrir jöfnum greiðslum

# Fasteign
verd: int = 30000000
fasteignamat: int = 25000000

In [2]:
# Afleiddar breytur - Ekki breyta

fjoldi_afborgana: int = 12 * lanstimi

In [3]:
# Innfluttningur á kóðasöfnum

from typing import Generator

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Reikniföll

def jafnar_afborganir(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    # verdtrygging: bool,
) -> Generator[tuple[int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur lán með jöfnum afborgunum.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - heildar greiðsla
        - stöðu höfuðstóls

    To-do: Útfæra verðtryggingu.
    '''
    afborgun_af_hofudstol = lansupphaed / fjoldi_afborgana
    hofudstoll = float(lansupphaed)
    for i in range(1, fjoldi_afborgana + 1):
        vaxtagreidsla = hofudstoll * ((vextir / 100) / 12)
        heildargreidsla = afborgun_af_hofudstol + vaxtagreidsla

        yield (
            i,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            int(round(heildargreidsla, 0)),
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol


def jafnar_greidslur(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    # verdtrygging: bool,
) -> Generator[tuple[int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur láns með jöfnum greidslum.

    Greiðslur inn á höfuðstól eru reiknaðar með formúlunni:

        M = H[r(1 + r)^n]/[(1 + r)^n – 1]

    þar sem:
        - M er mánaðarleg heildargreiðsla
        - H er höfuðstóll láns
        - r er vextir á mánuði (Vextir á ársgrundvelli / 12)
        - n er fjöldi afborgana (mánuðir)

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - heildar greiðsla
        - stöðu höfuðstóls

    To-do: Útfæra verðtryggingu.
    '''
    hofudstoll = float(lansupphaed)

    r = (vextir / 100) / 12
    n = fjoldi_afborgana
    heildargreidsla = lansupphaed * (r * (1 + r)**n) / ((1 + r)**n - 1)

    for i in range(1, fjoldi_afborgana + 1):
        vaxtagreidsla = hofudstoll * r
        afborgun_af_hofudstol = heildargreidsla - vaxtagreidsla

        yield (
            i,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            int(round(heildargreidsla, 0)),
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol



In [ ]:
# Útreikningur

jafnar_afborganir_overdtryggt = pd.DataFrame(
    jafnar_afborganir(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana,
        vextir=vextir,
        # verdtrygging=verdtrygging,
    ),
    columns=['Mánuður', 'Afborgun', 'Vextir', 'Heildargreiðsla' , 'Höfuðstóll'],
).set_index('Mánuður')

plt.plot(jafnar_afborganir_overdtryggt['Afborgun'], label='Afborgun')
plt.plot(jafnar_afborganir_overdtryggt['Vextir'], label='Vextir')
plt.plot(jafnar_afborganir_overdtryggt['Heildargreiðsla'], label='Heildargreiðsla')

plt.xlabel('Mánuður')
plt.ylabel('Upphæð (kr.)')
plt.title('Greiðslubyrði láns með jöfnum afborgunum')
plt.legend()

jafnar_greidslur_overdtryggt = pd.DataFrame(
    jafnar_greidslur(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana,
        vextir=vextir,
        # verdtrygging=verdtrygging,
    ),
    columns=['Mánuður', 'Afborgun', 'Vextir', 'Heildargreiðsla' , 'Höfuðstóll'],
).set_index('Mánuður')

plt.figure()
plt.plot(jafnar_greidslur_overdtryggt['Afborgun'], label='Afborgun')
plt.plot(jafnar_greidslur_overdtryggt['Vextir'], label='Vextir')
plt.plot(jafnar_greidslur_overdtryggt['Heildargreiðsla'], label='Heildargreiðsla')

plt.xlabel('Mánuður')
plt.ylabel('Upphæð (kr.)')
plt.title('Greiðslubyrði láns með jöfnum greiðslum')
plt.legend()